In [1]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os

import numpy as np

In [2]:
def display_image (path:str):
    img = Image.open(path)  
    plt.imshow(img)
    plt.axis('off') 
    plt.show()

### Handwritten Dataset Split

In [3]:
hand_df = pd.read_csv("../raw_data/cleaned_no_dupli_hand.csv")

In [4]:
hand_df.head(5)

,image_filename,line_text
0,3777_line_1.png,ዦክሃር ሳርናየቭ የተባለው የ 21 ዓመት ወጣት አሸባሪ ይህን ...
1,3777_line_2.png,ባፀደቀው ፍርድ ቤት ከትናንት በስቲያ በቀረበበት ወቅት እንደሆነ ...
2,3777_line_3.png,ለምሳሌ ያህል ፣ ልጆቻችሁ በትምህርት ቤት ጥሩ ውጤት ቢያመጡ የ...
3,3777_line_4.png,እንደሚሆኑ ይሰማቸዋል ?። የፓትርያርኩ በዓለ ሢመት ቀን በጾም ...
4,3777_line_5.png,ቸው ቀኑን በለውጥ ስለ መበየን በጽ / ቤታቸው የቀረበውን አጀንዳ...


In [5]:
hand_root = "../raw_data/extracted_handwritten_lines/"

In [20]:
display_image("/home/admin/Gabby/Amharic OCR/HandWritten_Amharic_English_OCR/raw_data/Amharic_Data/train/synthetic_image_10288_2.png")  
print(hand_df.iloc[0]['line_text'])

FileNotFoundError: [Errno 2] No such file or directory: '/home/admin/Gabby/Amharic OCR/HandWritten_Amharic_English_OCR/raw_data/Amharic_Data/train/synthetic_image_10288_2.png'

In [7]:
texts = hand_df['line_text'].tolist()

In [9]:
from collections import Counter
all_text = ''.join(texts)
char_counts = Counter(all_text)



In [10]:
sorted_counts = dict(sorted(char_counts.items(), key=lambda item: item[1], reverse=True))



In [11]:

texts = hand_df['line_text'].dropna().astype(str)
char_df = pd.DataFrame(char_counts.items(), columns=["char", "count"])

# Compute 95th percentile threshold
threshold = np.percentile(char_df['count'], 30)

# Define rare characters: strictly below 95th percentile
rare_chars = set(char_df[char_df['count'] < threshold]['char'])

# Tag rows that contain rare characters
def contains_rare_char(text):
    return any(char in rare_chars for char in text)

hand_df['has_rare'] = texts.apply(contains_rare_char)

# Calculate percentage
percentage = (hand_df['has_rare'].sum() / len(hand_df)) * 100
print(f"Percentage of entries with rare characters: {percentage:.2f}%")


Percentage of entries with rare characters: 14.77%


In [12]:

#seperate df with rare and non rare characters
hand_df_rare = hand_df[hand_df['has_rare'] == True]
hand_df_non_rare = hand_df[hand_df['has_rare'] == False]

In [13]:
len(hand_df_rare), len(hand_df_non_rare)

(6036, 34820)

In [14]:
hand_df_non_rare['type'] = 'handwritten'

/tmp/ipykernel_83660/3912901920.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hand_df_non_rare['type'] = 'handwritten'


In [15]:
hand_df_non_rare.head(5)

,image_filename,line_text,has_rare,type
2,3777_line_3.png,ለምሳሌ ያህል ፣ ልጆቻችሁ በትምህርት ቤት ጥሩ ውጤት ቢያመጡ የ...,False,handwritten
4,3777_line_5.png,ቸው ቀኑን በለውጥ ስለ መበየን በጽ / ቤታቸው የቀረበውን አጀንዳ...,False,handwritten
5,3777_line_6.png,የተለያዩ አማራጮችን አይቷል። አንዲት ወላድ ሴት ( ከአራስ ቤት...,False,handwritten
6,3777_line_7.png,በአንዋር መስጂድ ተገኝተሻል በመርካቶ አራተኛ ፖሊስ ጣቢያ አስ...,False,handwritten
7,3777_line_8.png,እያለ እያዩት አውሬዎቹ ፖሊሶቻችሁ ልጇን እንኳ እንዳታጠባ የከ...,False,handwritten


In [16]:
train_df = pd.DataFrame(columns=['image_filename', 'line_text', 'type'])

In [17]:
import os
import shutil
from PIL import Image
count = 0

for i in range(len(hand_df_non_rare)):
    source_file = (os.path.join(hand_root , hand_df_non_rare.iloc[i]['image_filename']))  
    destination_file = os.path.join("../raw_data/Amharic_Data/train", ("hand_" + hand_df_non_rare.iloc[i]['image_filename']))
    
    # if os.path.isfile(source_file):
    #     shutil.copy2(source_file, destination_file)
    # # display_image(destination_file)
    # print(hand_df_rare.iloc[i]['line_text'])
    # print(hand_df_rare.iloc[i]['has_rare'])
    # print(hand_df_rare.iloc[i]['type'])
    # print("--------------------------------------------------")
    
    new_row = pd.DataFrame({'image_filename': ["handwritten_" + hand_df_non_rare.iloc[i]['image_filename']], 
                        'line_text': [hand_df_non_rare.iloc[i]['line_text']], 
                        'type': ['handwritten']})

    # Use pd.concat to append the new row to the DataFrame
    train_df = pd.concat([train_df, new_row], ignore_index=True)
   

In [19]:
train_df.to_csv("../raw_data/Amharic_Data/train/hand_train.csv", index=False)

In [18]:
len(train_df)

34820